In [15]:
from lambeq import BobcatParser
import numpy as np
from lambeq.ansatz import IQPAnsatz
from lambeq import AtomicType, NumpyModel
from discopro.anaphora import connect_anaphora
parser = BobcatParser()

In [28]:
sent1 = 'Alice loves Bob.'
sent2 = 'He loves her.'

from discopro.grammar import tensor
diag1 = parser.sentence2diagram(sent1)
diag2 = parser.sentence2diagram(sent2)
diag = tensor(diag1, diag2)
diag = connect_anaphora(diag, 3, 2)

In [29]:
# Define atomic types
N = AtomicType.NOUN
S = AtomicType.SENTENCE

# Convert string diagram to quantum circuit
ansatz = IQPAnsatz({N: 1, S: 1}, n_layers=2)

In [30]:
def convert(pr_dist):
    """
    Convert a empirical table with the usual
    ab, aB, Ab, AB ordering to the cyclic 
    ab, bA, AB, Ba ordering.
    """
    new_pr_dist = np.zeros_like(pr_dist)
    new_pr_dist[0] = pr_dist[0] 
    new_pr_dist[1] = pr_dist[2][[0,2,1,3]]
    new_pr_dist[2] = pr_dist[3]
    new_pr_dist[3] = pr_dist[1][[0,2,1,3]]
    return new_pr_dist

In [79]:
## from lambeq import NumpyModel
from lambeq.backend.quantum import Rx, Rz, Ry
from sympy.core.symbol import Symbol
from contextuality import Model, chsh_scenario
base_circ = ansatz(diag)

circs = []
for i in range(2):
    for j in range(2):
        circ = base_circ
        if i:
            #circ = circ.apply_gate(Rx(Symbol('x0')), 0)
            #circ = circ.apply_gate(Rz(Symbol('z0')), 0)
            circ = circ.apply_gate(Ry(0), 0)
        else:
            circ = circ.apply_gate(Ry(np.pi/4), 0)
        if j:
            #circ = circ.apply_gate(Rx(Symbol('x1')), 1)
            #circ = circ.apply_gate(Rz(Symbol('z1')), 1)
            circ = circ.apply_gate(Ry(np.pi/8), 1)
        else:
            circ = circ.apply_gate(Ry(3*np.pi/8), 1)
        circs.append(circ)

model = NumpyModel.from_diagrams(circs)
model.initialise_weights()

pr_dist = model.get_diagram_output(circs)
pr_dist = np.reshape(pr_dist, (4,4))

scenario = chsh_scenario()
# emp = Model(scenario, pr_dist)
emp = Model(scenario, convert(pr_dist))
print(f'CF: {emp.contextual_fraction():.10f}')
print(f'SF: {emp.signalling_fraction():.10f}')
print(f'CbD: {emp.CbD_measure():.10f}')
print(f'DI: {emp.CbD_direct_influence():.10f}')

CF: -0.0000000001
SF: -0.0000000002
CbD: -0.5720710297
DI: 0.0000000000
